In [1]:
!kaggle datasets download -d rashikrahmanpritom/plant-disease-recognition-dataset

Dataset URL: https://www.kaggle.com/datasets/rashikrahmanpritom/plant-disease-recognition-dataset
License(s): CC0-1.0
100% 1.24G/1.25G [00:11<00:00, 137MB/s]
100% 1.25G/1.25G [00:11<00:00, 117MB/s]


In [2]:
!unzip /content/plant-disease-recognition-dataset.zip

Archive:  /content/plant-disease-recognition-dataset.zip
  inflating: Test/Test/Healthy/8ddaa5a5caa5caa8.jpg  
  inflating: Test/Test/Healthy/8ddaac1bd6c8cd0a.jpg  
  inflating: Test/Test/Healthy/8ddd5ec1c0de38c4.jpg  
  inflating: Test/Test/Healthy/8def3f60308ab41b.jpg  
  inflating: Test/Test/Healthy/8def4d91382175c3.jpg  
  inflating: Test/Test/Healthy/8df452e2e38c0b6e.jpg  
  inflating: Test/Test/Healthy/8dfae9d78cc32089.jpg  
  inflating: Test/Test/Healthy/8e3dbccdfe08c850.jpg  
  inflating: Test/Test/Healthy/8e68163c62dc57d5.jpg  
  inflating: Test/Test/Healthy/8e6a823cce9ff40c.jpg  
  inflating: Test/Test/Healthy/8e77857194a59a87.jpg  
  inflating: Test/Test/Healthy/8e79802b3fb770c8.jpg  
  inflating: Test/Test/Healthy/8e7986d1ecd36445.jpg  
  inflating: Test/Test/Healthy/8e7e70a6878c1c79.jpg  
  inflating: Test/Test/Healthy/8e82b1a51bd11afe.jpg  
  inflating: Test/Test/Healthy/8e8470687be37378.jpg  
  inflating: Test/Test/Healthy/8e858c8397706b7b.jpg  
  inflating: Test/Test/He

In [3]:
import tensorflow as tf

In [4]:
train_dir = '/content/Test/Test'
test_dir = '/content/Train/Train'
val_dir = '/content/Validation/Validation'

In [5]:
train = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = ['Healthy', 'Powdery', 'Rust'],
    batch_size = 16,
    image_size = (256, 256),
    shuffle = True,
    seed = 42,
    validation_split = 0,
    crop_to_aspect_ratio = True
)

Found 150 files belonging to 3 classes.


In [6]:
test = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = ['Healthy', 'Powdery', 'Rust'],
    batch_size = 16,
    image_size = (256, 256),
    shuffle = True,
    seed = 42,
    validation_split = 0,
    crop_to_aspect_ratio = True
)

Found 1322 files belonging to 3 classes.


In [7]:
validation = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = ['Healthy', 'Powdery', 'Rust'],
    batch_size = 16,
    image_size = (256, 256),
    shuffle = True,
    seed = 42,
    validation_split = 0,
    crop_to_aspect_ratio = True
)

Found 60 files belonging to 3 classes.


In [8]:
from tensorflow.keras.layers import Rescaling

In [9]:
scaler = Rescaling(1./255)

train = train.map(lambda x, y: (scaler(x), y))
test = test.map(lambda x, y: (scaler(x), y))
validation = validation.map(lambda x, y: (scaler(x), y))

In [10]:
augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomRotation(
        factor = (-.25, .3),
        fill_mode = 'reflect',
        interpolation = 'bilinear',
        seed = 42),

        tf.keras.layers.RandomBrightness(
        factor = (-.45, .45),
        value_range = (0.0, 1.0),
        seed = 42),

        tf.keras.layers.RandomContrast(
        factor = (.5),
        seed = 42)
    ]
)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization

In [12]:
model = Sequential()

model.add(augmentation)

model.add(Conv2D(32, (3,3), strides = 1, padding = 'same', input_shape = (256, 256, 3)))

model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv2D(64, (5,5), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Dropout(0.3))

model.add(Conv2D(256, (5,5), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Dropout(0.3))

model.add(Conv2D(512, (3,3), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(2048))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation = 'softmax'))

In [13]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [15]:
early_stopping = EarlyStopping(monitor = 'val_accuracy',
                              patience = 5, mode = 'max',
                              restore_best_weights = True)

checkpoint = ModelCheckpoint('best_model.h5',
                            monitor = 'val_accuracy',
                            save_best_only = True)

In [19]:
model.fit(train,
          epochs=30,
          validation_data=validation,
          batch_size=16)

Epoch 1/30
10/10 [==============================] - 13s 702ms/step - loss: 1.0224 - accuracy: 0.7600 - val_loss: 16.4576 - val_accuracy: 0.3333
Epoch 2/30
10/10 [==============================] - 10s 403ms/step - loss: 0.8893 - accuracy: 0.7933 - val_loss: 16.5969 - val_accuracy: 0.3333
Epoch 3/30
10/10 [==============================] - 11s 699ms/step - loss: 0.7778 - accuracy: 0.8467 - val_loss: 15.5133 - val_accuracy: 0.3333
Epoch 4/30
10/10 [==============================] - 9s 385ms/step - loss: 0.7553 - accuracy: 0.7800 - val_loss: 11.8881 - val_accuracy: 0.3500
Epoch 5/30
10/10 [==============================] - 10s 404ms/step - loss: 1.0232 - accuracy: 0.7933 - val_loss: 13.4900 - val_accuracy: 0.3333
Epoch 6/30
10/10 [==============================] - 10s 384ms/step - loss: 0.6656 - accuracy: 0.8200 - val_loss: 20.4007 - val_accuracy: 0.3333
Epoch 7/30
10/10 [==============================] - 10s 511ms/step - loss: 0.6381 - accuracy: 0.8333 - val_loss: 18.3728 - val_accuracy: 

In [22]:
model.save('model.h5')